Uploading Phase:

In [ ]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy
!pip install scikit-learn

Calling the Neccessary Libraries:

In [ ]:
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from skimage import exposure
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import cv2
import glob
import os

Finding Height and Width of an Image:

In [ ]:
sample_image_path = '/content/drive/MyDrive/FACE IMAGES/1.1.pgm'

# Load the image in grayscale
img = cv2.imread('/content/drive/MyDrive/FACE IMAGES/1.1.pgm', cv2.IMREAD_GRAYSCALE)

# Print its shape
print("Image shape:", img.shape)

height = img.shape[0]
print("Height:", height)

Image shape: (80, 64)
Height: 80


Training Part:

In [ ]:
import os
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import normalize

# ----------------------------
# CONFIGURATION
# ----------------------------
DATASET_PATH = "/content/drive/MyDrive/FACE IMAGES"
IMAGE_EXTENSION = ".pgm"
NUM_SUBJECTS = 250
TRAIN_PER_SUBJECT = 2
PATCH_SIZE = 5
stride = 1

# Use higher resolution
target_size = (90, 120)

# LBP settings (change here only)
P = 16  # Number of sampling points
R_values = [1, 2]  # Radii to use

# ----------------------------
# FUNCTION: Get binary version of LBP matrix (5×5 of bit-strings)
# ----------------------------
def decimal_matrix_to_binary_strings(matrix, bits):
    return np.array([[format(val, f'0{bits}b') for val in row] for row in matrix])

# ----------------------------
# FUNCTION: Find min-rotated binary matrix (lexical minimum)
# ----------------------------
def rotate_binary_matrix_to_min(bin_matrix):
    def rotate_all_versions(m):
        return [
            m,
            np.rot90(m, 1),
            np.rot90(m, 2),
            np.rot90(m, 3),
            np.fliplr(m),
            np.flipud(m),
            np.fliplr(np.rot90(m, 1)),
            np.flipud(np.rot90(m, 1))
        ]
    rotations = rotate_all_versions(bin_matrix)
    min_bin_matrix = min(rotations, key=lambda x: x.tobytes())
    return min_bin_matrix

# ----------------------------
# FUNCTION: Convert binary string matrix back to decimal matrix
# ----------------------------
def binary_strings_to_decimal_matrix(bin_matrix):
    return np.array([[int(val, 2) for val in row] for row in bin_matrix], dtype=np.uint16)

# ----------------------------
# FUNCTION: Create RIU2 mapping table
# ----------------------------
def get_riu2_mapping(P):
    table = np.zeros(2**P, dtype=np.uint8)
    for i in range(2**P):
        binary = [(i >> j) & 1 for j in range(P)]
        binary.append(binary[0])
        transitions = sum(binary[j] != binary[j+1] for j in range(P))
        if transitions <= 2:
            table[i] = sum(binary[:P])
        else:
            table[i] = P + 1
    return table

# ----------------------------
# FUNCTION: Extract LBP histogram with full pipeline
# ----------------------------
def extract_rotated_lbp_histogram(block, P, R, riu2_map):
    lbp_block = local_binary_pattern(block, P, R, method='ror').astype(np.uint16)
    bin_matrix = decimal_matrix_to_binary_strings(lbp_block, bits=P)
    bin_matrix_min = rotate_binary_matrix_to_min(bin_matrix)
    dec_matrix_min = binary_strings_to_decimal_matrix(bin_matrix_min)
    lbp_block_uniform = riu2_map[dec_matrix_min]
    hist, _ = np.histogram(
        lbp_block_uniform.ravel(),
        bins=np.arange(0, P + 3),
        density=True
    )
    return hist, lbp_block_uniform

# ----------------------------
# MAIN TRAINING LOOP
# ----------------------------
riu2_map = get_riu2_mapping(P)
train_lbp_features = []
train_lbp_raw_codes = []  # Optional: store raw codes if needed for debugging or visualization

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_id in range(1, TRAIN_PER_SUBJECT + 1):
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Could not load image: {filename}")
            continue

        resized = cv2.resize(img, target_size)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        resized_eq = clahe.apply(resized).astype(np.float64) / 255.0
        resized_norm = (resized_eq - np.mean(resized_eq)) / np.std(resized_eq)

        feature_vector = []
        raw_code_blocks = []
        for y in range(0, resized.shape[0] - PATCH_SIZE + 1, 2):
            for x in range(0, resized.shape[1] - PATCH_SIZE + 1, 2):
                block = resized_norm[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                combined_hist = []
                for R in R_values:
                    hist, raw_codes = extract_rotated_lbp_histogram(block, P=P, R=R, riu2_map=riu2_map)
                    combined_hist.extend(hist.tolist())
                feature_vector.extend(combined_hist)

        train_lbp_features.append(feature_vector)
        print(f"✅ Processed {filename} | Feature length: {len(feature_vector)}")

# ----------------------------
# CONVERT TO NUMPY ARRAY + NORMALIZE
# ----------------------------
train_lbp_features = np.array(train_lbp_features, dtype=np.float32)
train_lbp_features = normalize(train_lbp_features, norm='l2')

# ----------------------------
# FINAL OUTPUT
# ----------------------------
print("\n✅ All training features extracted using CLAHE + Multi-R LBP + RIU2")
print("🔢 Feature matrix shape:", train_lbp_features.shape)


✅ Processed 1.1.pgm | Feature length: 89784
✅ Processed 1.2.pgm | Feature length: 89784
✅ Processed 2.1.pgm | Feature length: 89784
✅ Processed 2.2.pgm | Feature length: 89784
✅ Processed 3.1.pgm | Feature length: 89784
✅ Processed 3.2.pgm | Feature length: 89784
✅ Processed 4.1.pgm | Feature length: 89784
✅ Processed 4.2.pgm | Feature length: 89784
✅ Processed 5.1.pgm | Feature length: 89784
✅ Processed 5.2.pgm | Feature length: 89784
✅ Processed 6.1.pgm | Feature length: 89784
✅ Processed 6.2.pgm | Feature length: 89784
✅ Processed 7.1.pgm | Feature length: 89784
✅ Processed 7.2.pgm | Feature length: 89784
✅ Processed 8.1.pgm | Feature length: 89784
✅ Processed 8.2.pgm | Feature length: 89784
✅ Processed 9.1.pgm | Feature length: 89784
✅ Processed 9.2.pgm | Feature length: 89784
✅ Processed 10.1.pgm | Feature length: 89784
✅ Processed 10.2.pgm | Feature length: 89784
✅ Processed 11.1.pgm | Feature length: 89784
✅ Processed 11.2.pgm | Feature length: 89784
✅ Processed 12.1.pgm | Featu

Testing Part:

In [ ]:
# ----------------------------
# TEST FEATURE EXTRACTION (Images 3 and 4)
# ----------------------------
test_lbp_features = []
test_labels = []

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_id in range(3, 5):  # Using images 3 and 4 for testing
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Could not load test image: {filename}")
            continue

        resized = cv2.resize(img, target_size)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        resized_eq = clahe.apply(resized).astype(np.float64) / 255.0
        resized_norm = (resized_eq - np.mean(resized_eq)) / np.std(resized_eq)

        feature_vector = []
        for y in range(0, resized.shape[0] - PATCH_SIZE + 1, 2):
            for x in range(0, resized.shape[1] - PATCH_SIZE + 1, 2):
                block = resized_norm[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                combined_hist = []
                for R in R_values:
                    hist, _ = extract_rotated_lbp_histogram(block, P=P, R=R, riu2_map=riu2_map)
                    combined_hist.extend(hist.tolist())
                feature_vector.extend(combined_hist)

        test_lbp_features.append(feature_vector)
        test_labels.append(subject_id)
        print(f"✅ Processed test image: {filename}")

# ----------------------------
# FINALIZE TEST ARRAYS
# ----------------------------
test_lbp_features = np.array(test_lbp_features, dtype=np.float32)
test_lbp_features = normalize(test_lbp_features, norm='l2')
test_labels = np.array(test_labels)

print("\n✅ All test features extracted using CLAHE + Multi-R LBP + Binary Matrix Rotation + RIU2!")
print("🧪 Test feature matrix shape:", test_lbp_features.shape)
print("🎯 Test labels shape:", test_labels.shape)


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


✅ Processed test image: 1.3.pgm
✅ Processed test image: 1.4.pgm
✅ Processed test image: 2.3.pgm
✅ Processed test image: 2.4.pgm
✅ Processed test image: 3.3.pgm
✅ Processed test image: 3.4.pgm
✅ Processed test image: 4.3.pgm
✅ Processed test image: 4.4.pgm
✅ Processed test image: 5.3.pgm
✅ Processed test image: 5.4.pgm
✅ Processed test image: 6.3.pgm
✅ Processed test image: 6.4.pgm
✅ Processed test image: 7.3.pgm
✅ Processed test image: 7.4.pgm
✅ Processed test image: 8.3.pgm
✅ Processed test image: 8.4.pgm
✅ Processed test image: 9.3.pgm
✅ Processed test image: 9.4.pgm
✅ Processed test image: 10.3.pgm
✅ Processed test image: 10.4.pgm
✅ Processed test image: 11.3.pgm
✅ Processed test image: 11.4.pgm
✅ Processed test image: 12.3.pgm
✅ Processed test image: 12.4.pgm
✅ Processed test image: 13.3.pgm
✅ Processed test image: 13.4.pgm
✅ Processed test image: 14.3.pgm
✅ Processed test image: 14.4.pgm
✅ Processed test image: 15.3.pgm
✅ Processed test image: 15.4.pgm
✅ Processed test image: 16.3

Recognition and Accuracy:

In [ ]:
# -------------------------
# RECOGNITION + ACCURACY (Manhattan Distance)
# -------------------------
correct_matches = 0
total_tests = len(test_labels)

for idx, test_feat in enumerate(test_lbp_features):
    # Step 1: Compute Manhattan (L1) distance from all training samples
    distances = np.sum(np.abs(train_lbp_features - test_feat), axis=1)

    # Step 2: Find best match
    best_match_idx = np.argmin(distances)

    # Step 3: Determine predicted label
    predicted_label = (best_match_idx // TRAIN_PER_SUBJECT) + 1
    true_label = test_labels[idx]

    # Step 4: Match check
    match_result = "✅" if predicted_label == true_label else "❌"
    print(f"Test #{idx + 1} | True: {true_label} | Predicted: {predicted_label} {match_result}")

    if predicted_label == true_label:
        correct_matches += 1

# Final accuracy
accuracy = (correct_matches / total_tests) * 100
print(f"\n🎯 Recognition Accuracy (Manhattan Distance): {accuracy:.2f}% ({correct_matches}/{total_tests})")


Test #1 | True: 1 | Predicted: 10 ❌
Test #2 | True: 1 | Predicted: 1 ✅
Test #3 | True: 2 | Predicted: 79 ❌
Test #4 | True: 2 | Predicted: 2 ✅
Test #5 | True: 3 | Predicted: 3 ✅
Test #6 | True: 3 | Predicted: 3 ✅
Test #7 | True: 4 | Predicted: 4 ✅
Test #8 | True: 4 | Predicted: 4 ✅
Test #9 | True: 5 | Predicted: 5 ✅
Test #10 | True: 5 | Predicted: 5 ✅
Test #11 | True: 6 | Predicted: 6 ✅
Test #12 | True: 6 | Predicted: 6 ✅
Test #13 | True: 7 | Predicted: 7 ✅
Test #14 | True: 7 | Predicted: 7 ✅
Test #15 | True: 8 | Predicted: 8 ✅
Test #16 | True: 8 | Predicted: 8 ✅
Test #17 | True: 9 | Predicted: 9 ✅
Test #18 | True: 9 | Predicted: 9 ✅
Test #19 | True: 10 | Predicted: 10 ✅
Test #20 | True: 10 | Predicted: 10 ✅
Test #21 | True: 11 | Predicted: 230 ❌
Test #22 | True: 11 | Predicted: 104 ❌
Test #23 | True: 12 | Predicted: 12 ✅
Test #24 | True: 12 | Predicted: 12 ✅
Test #25 | True: 13 | Predicted: 13 ✅
Test #26 | True: 13 | Predicted: 13 ✅
Test #27 | True: 14 | Predicted: 24 ❌
Test #28 | True